<img src="https://i.ibb.co/K5n24Nd/1.png"  alt="KCode Banner"  />

<a href="https://www.linkedin.com/in/kauebr/"><img src="https://cdn-icons-png.flaticon.com/512/174/174857.png" width="30" ></a> <a href="mailto: kauemandarino@gmail.com"><img src="https://cdn-icons-png.flaticon.com/512/5968/5968534.png" width="30" ></a><a href="
https://api.whatsapp.com/send?phone=5541991835800&text=Oi"> <img src="https://imagepng.org/wp-content/uploads/2017/08/whatsapp-icone-1.png" width="30" ></a><a href="https://github.com/kauebr/"><img src="https://github.githubassets.com/images/modules/logos_page/Octocat.png" width="30" ></a>


# Análise exploratório de dados

## Objetivo:
* Analisar os dados levantados e validar hipóteses

## Hipóteses 

* Investir nas ações rendeu mais que investir na Selic

qual foram as taxas de rendimento? quais foram as taxas de rendimento Selic

* Estou fazendo:
monta um dicionário com lista de dataframes com os csvs das listas e da selic
de preferencia puxa esses dataframes com url

da uma olhada no minhas notas e no meu google colab que ennsian a fazer isso

* calcula a váriacao de cada dataframe pra ver se a variacao foi maior da selic ou das acoes, calcula tbm a média das acoes e a variaçao da média
tenta alguma lib q faça isso senao segue o que eu fiz para plota
estuda estuda


* a variação que for maior é a que mais rendeu


In [62]:
import pandas as pd

### Modelando